In [ ]:
import bz2
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Read market orders bz2 file into a dataframe
with bz2.open("data/market-orders-latest.v3.csv.bz2", "rt") as f:
    orders_df = pd.read_csv(f)
orders_df["type_id"] = orders_df["type_id"].astype(int)
orders_df["region_id"] = orders_df["region_id"].astype(int)
orders_df

In [ ]:

types_pkl = "data/types_df.pkl"
with open(types_pkl, "rb") as f:
    types_df = pickle.load(f)
types_df["type_id"] = types_df["type_id"].astype(int)
types_df

In [ ]:
regions_pkl = "data/regions_df.pkl"
with open(regions_pkl, "rb") as f:
    regions_df = pickle.load(f)
regions_df["region_id"] = regions_df["region_id"].astype(int)
regions_df

In [ ]:
structures_df = pd.read_json("data/structures-latest.v2.json", orient="index")
structures_df["structure_id"] = structures_df["structure_id"].astype(int)
# structures_df["type_id"] = structures_df["type_id"].astype(int)
# structures_df["owner_id"] = structures_df["owner_id"].astype(int)
# structures_df = structures_df[
#     (structures_df["is_market_structure"] == True)
#     & (structures_df["is_public_structure"] == True)
# ]
structures_df

In [ ]:
df_stations = pd.read_csv("data/stations.csv")
df_stations["station_id"] = df_stations["station_id"].astype(int)
df_stations

In [ ]:
orders_with_desc = orders_df.merge(types_df, how="inner", on="type_id")
orders_with_desc = orders_with_desc.merge(regions_df, how="inner", on="region_id")
orders_with_desc = orders_with_desc.merge(df_stations, how="inner", left_on="station_id", right_on="station_id")
orders_with_desc["station_id"] = orders_with_desc["station_id"].astype(int)
orders_with_desc

In [ ]:
# get mapStargates.jsonl from the data folder
stargates_df = pd.read_json("data/mapStargates.jsonl", lines=True)
stargates_df = stargates_df[['_key', 'destination', 'position', 'solarSystemID', 'typeID']]
stargates_df

In [ ]:
# get mapSolarSystems.jsonl from the data folder
solar_systems_df = pd.read_json("data/mapSolarSystems.jsonl", lines=True)
solar_systems_df = solar_systems_df[
    [
        '_key', 
        'border', 
        'constellationID', 
        'hub', 
        'international',
        'luminosity', 
        'name', 
        'planetIDs', 
        'position', 
        'position2D', 
        'radius',
        'regionID', 
        'regional', 
        'securityClass', 
        'securityStatus', 
        'starID',
        'stargateIDs', 
        'corridor', 
        'fringe', 
        'wormholeClassID', 
        'visualEffect',
        'disallowedAnchorCategories', 
        'disallowedAnchorGroups', 
        'factionID'
    ]
]
solar_systems_df["name"] = [x["en"] for x in solar_systems_df["name"]]
solar_systems_df

In [ ]:
system_name_dict = solar_systems_df[["_key", "name"]].set_index("_key")["name"].to_dict()
system_name_dict

In [ ]:
orders_with_desc["system_name"] = orders_with_desc["system_id"].map(system_name_dict)
orders_with_desc["system_name"]

In [ ]:
npc_stations_df = pd.read_json("data/npcStations.jsonl", lines=True)
npc_stations_df = npc_stations_df[[
    "_key",
    "solarSystemID",
]]
npc_stations_df["system_name"] = npc_stations_df["solarSystemID"].map(system_name_dict)
npc_stations_df = npc_stations_df.rename(columns={"_key": "station_id", "solarSystemID": "system_id"})
npc_stations_df

In [ ]:
# create a graph of the stargates
G = nx.DiGraph()
for _, row in stargates_df[stargates_df["position"].notna()].iterrows():
    G.add_node(system_name_dict[row['solarSystemID']], position=row['position'])
    G.add_edge(system_name_dict[row['solarSystemID']], system_name_dict[row['destination']["solarSystemID"]])
list(G.edges)[:5]

In [ ]:
# calculate shortest path between two systems
source_system = "Kino"
target_system = "Arvasaras"
shortest_path = nx.shortest_path(G, source=source_system, target=target_system)
shortest_path

In [ ]:
# Buy order aggregation by system
orders_with_desc[orders_with_desc["is_buy_order"] == True].groupby("system_name").agg({
    "order_id": "count",
    "price": ["mean", "median", "min", "max"],
    "volume_remain": "sum"
}).sort_values(by=("order_id", "count"), ascending=False).head(20)

In [ ]:
# Sell order aggregation by system
orders_with_desc[orders_with_desc["is_buy_order"] == False].groupby("system_name").agg({
    "order_id": "count",
    "price": ["mean", "median", "min", "max"],
    "volume_remain": "sum"
}).sort_values(by=("order_id", "count"), ascending=False).head(20)

In [ ]:
MAX_VOLUME = 5737
cols = [
    "type_id",
    "type_name",
    "station_id",
    "station_name",
    "system_id",
    "system_name",
    "region_id",
    "region_name",
    "price",
    "volume_remain",    
]
buy_orders = orders_with_desc[orders_with_desc["is_buy_order"] == True][cols + ["packaged_volume"]]
sell_orders = orders_with_desc[orders_with_desc["is_buy_order"] == False][cols]
buy_sell = buy_orders.merge(
    sell_orders,
    how="inner",
    on=["type_id"],
    suffixes=("_buy", "_sell"),
).dropna()

buy_sell = buy_sell[(buy_sell["packaged_volume"] < MAX_VOLUME) & (buy_sell["packaged_volume"] > 0)]
buy_sell["spread"] = buy_sell["price_buy"] - buy_sell["price_sell"]
buy_sell["max_units"] = MAX_VOLUME / buy_sell["packaged_volume"]
buy_sell["max_units"] = buy_sell["max_units"].astype(int)
buy_sell["max_units"] = buy_sell["volume_remain_buy"].clip(upper=buy_sell["volume_remain_sell"]).clip(upper=buy_sell["max_units"])
# total profit should be spread times the minimum of the 3 volumes, buy sell and max units
buy_sell["total_profit"] = buy_sell["spread"] * buy_sell["max_units"] / 1e6
buy_sell["spread_pct"] = buy_sell["spread"] / buy_sell["price_sell"] * 100
buy_sell = buy_sell[(buy_sell["spread"] > 0) & (buy_sell["spread_pct"] > 10) & (buy_sell["total_profit"] > 25)]
buy_sell = buy_sell.sort_values(by="spread_pct", ascending=False)
buy_sell.head(20)

In [ ]:
def _get_path(row):
    if row["region_name_sell"] != "Pochven" and row["region_name_buy"] == "Pochven":
        # Filament into pochven in 1 jump
        return 1
    try:
        if row["region_name_sell"] == "Pochven" and row["region_name_buy"] != "Pochven":
            # Assume we filament out of Pochven near Jita. 
            path = nx.shortest_path(G, source="Jita", target=row["system_name_buy"])
        else:
            path = nx.shortest_path(G, source=row["system_name_sell"], target=row["system_name_buy"])
        return len(path) - 1
    except nx.NetworkXNoPath:
        return None
buy_sell["jumps"] = buy_sell.apply(_get_path, axis=1)
buy_sell["profit_per_jump"] = buy_sell["total_profit"] / buy_sell["jumps"]
buy_sell.head(20)

In [ ]:
buy_sell.sort_values(by="spread_pct", ascending=False)[[
    "type_name_buy",
    "station_name_buy",
    "station_name_sell",
    "price_buy",
    "price_sell",
    "max_units",
    "jumps",
    "total_profit",
    "spread_pct",
    "profit_per_jump",
]].head(20)